In [1]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from typing import TypedDict
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from typing import Literal

In [2]:
load_dotenv()
llm = HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.3',
    task='chat-completion',
    max_new_tokens=50
)

model = ChatHuggingFace(llm = llm)

/Users/suryanshthakur/Desktop/Personal Projects/GenAIApps/SmallProjects/lvenev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:
    #fetch title 
    title = state['title']
    
    # call llm gen outline 
    prompt = f'Generate a detailed outline for a blog on the topiv - {title}'
    outline  = model.invoke(prompt).content
    
    # update state
    state['outline'] = outline
    return state

def create_blog(state: BlogState ) -> BlogState:
    title = state['title']
    outline = state['outline']
    
    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'
    content = model.invoke(prompt).content
    # update state
    state['content'] = content
    return state
    

In [5]:
graph  = StateGraph(BlogState)

# nodes 
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

# edges 
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()


In [ ]:
initial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(initial_state)

